In [6]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [7]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

candidate_labels = ['Healthcare and Pharmaceuticals', 'Technology', 'Energy', 'Financials', 'Automotive', 'Agriculture', 'Airlines and Travel', 'Cryptocurrency and Fintech', 'Defense and Aerospace']

Device set to use mps:0


In [8]:
def classify_tweet(text):
    try:
        result = classifier(text, candidate_labels, multi_label=True)

        return dict(zip(result['labels'], result['scores']))
    except Exception as e:
        print(f'Error classifying tweet: {e}')
        return {}

In [17]:
tweets = pd.read_csv('../data/tweets_with_sentiment.csv')
tweets.head()

,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score
0,1603456812723933194,12.0,jack,"RT @ODELL: @nobsbitcoin love to see it, if yal...",en,2022-12-15 18:27:49.597000+00:00,Neutral,0.977740
1,1600412216687149056,12.0,jack,RT @TheGuySwann: It would be so amazing to see...,en,2022-12-07 08:49:41.336000+00:00,Positive,0.999987
2,1595502702854692866,12.0,jack,RT @kendricklamar: big steppers tour (direct...,en,2022-11-23 19:41:02.024000+00:00,Neutral,0.999613
3,1589450807216599040,12.0,jack,@litcapital @elonmusk Sure,en,2022-11-07 02:52:57.665000+00:00,Neutral,0.985752
4,1582442966521503745,12.0,jack,RT @bluesky: Bluesky is building a social prot...,en,2022-10-18 18:46:18.244000+00:00,Neutral,0.999923


In [18]:
tqdm.pandas()
tweets['topic_scores'] = tweets['clean_text'].progress_apply(classify_tweet)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5059/5059 [22:12<00:00,  3.80it/s]


In [19]:
tweets['top_topic'] = tweets['topic_scores'].apply(lambda d: max(d, key=d.get) if d else None)

tweets = pd.concat([tweets, tweets['topic_scores'].apply(pd.Series)], axis=1)
del tweets['topic_scores']

In [20]:
tweets.head()

,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score,top_topic,Cryptocurrency and Fintech,Technology,Healthcare and Pharmaceuticals,Defense and Aerospace,Financials,Airlines and Travel,Energy,Agriculture,Automotive
0,1603456812723933194,12.0,jack,"RT @ODELL: @nobsbitcoin love to see it, if yal...",en,2022-12-15 18:27:49.597000+00:00,Neutral,0.977740,Cryptocurrency and Fintech,0.757990,0.376340,0.045903,0.032895,0.019666,0.019118,0.013074,0.010588,0.007226
1,1600412216687149056,12.0,jack,RT @TheGuySwann: It would be so amazing to see...,en,2022-12-07 08:49:41.336000+00:00,Positive,0.999987,Cryptocurrency and Fintech,0.868653,0.853358,0.025603,0.044147,0.210190,0.011476,0.110531,0.037469,0.007175
2,1595502702854692866,12.0,jack,RT @kendricklamar: big steppers tour (direct...,en,2022-11-23 19:41:02.024000+00:00,Neutral,0.999613,Technology,0.012002,0.252524,0.057174,0.072260,0.034317,0.007996,0.006217,0.003851,0.017827
3,1589450807216599040,12.0,jack,@litcapital @elonmusk Sure,en,2022-11-07 02:52:57.665000+00:00,Neutral,0.985752,Technology,0.284053,0.768173,0.263169,0.119607,0.364800,0.058842,0.239539,0.033245,0.125718
4,1582442966521503745,12.0,jack,RT @bluesky: Bluesky is building a social prot...,en,2022-10-18 18:46:18.244000+00:00,Neutral,0.999923,Technology,0.000134,0.058334,0.002962,0.001737,0.000540,0.004032,0.008653,0.003117,0.000706


In [22]:
tweets.to_csv('../data/tweets_with_sentiment_and_topic.csv')